https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb

In [4]:
# uncomment before running on google colab 
!pip install transformers==4.8.1
!pip install datasets 

  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.17
    Uninstalling huggingface-hub-0.0.17:
      Successfully uninstalled huggingface-hub-0.0.17
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 1.12.1 requires huggingface-hub<0.1.0,>=0.0.14, but you have huggingface-hub 0.0.12 which is incompatible.


  Using cached huggingface_hub-0.0.17-py3-none-any.whl (52 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.12
    Uninstalling huggingface-hub-0.0.12:
      Successfully uninstalled huggingface-hub-0.0.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.8.1 requires huggingface-hub==0.0.12, but you have huggingface-hub 0.0.17 which is incompatible.


# Imports 

In [5]:
from datasets import load_dataset, load_metric 
from transformers import AutoTokenizer,AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator
import numpy as np
import torch


max_length=512 
doc_stride= 128


In [6]:
# load dataset 
datasets = load_dataset("squad_v2")

Downloading:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/9.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:

def prepare_train_features(examples):
    ''' given examples from Squad dataset: 
     accounts for examples longer then 512 tokens 
    returns: tokenized examples with: input ids, attention_mask, answer start position index, answer end position index''' 

    # tokenize examples accounting for some of them being to long to fit in a single feature 
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # get mapping from features to corresponding example in dataset  
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    #  get offset_mapping to map tokens to character position in original context  
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # add keys for start_positions and end_positions of answers to tokenized_examples 
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    # iterate through all offset_mappings (the corresponding start and end character in the original text that gave our token.)
    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i] # input ids for tokenized examples 
        #cls_index = input_ids.index(tokenizer.cls_token_id)         # get cls index to label impossible answers 


        # Grab the sequence  corresponding for example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans,  
        sample_index = sample_mapping[i] # index of the example containing this span of text.
        answers = examples["answers"][sample_index] # get answers for this example 
        # If no answers are given for this example set the answer start and end position to 0 
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(0)
            tokenized_examples["end_positions"].append(0)
        else: # answers are given 
            # get start and end character index of the answer in the context.
            start_char_index = answers["answer_start"][0]
            end_char_index = start_char_index + len(answers["text"][0])

            # get start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # get end token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # if the answer is out of the span 
            if not (offsets[token_start_index][0] <= start_char_index and offsets[token_end_index][1] >= end_char_index):
              #  set start and end position for answer to 0 since not in this span
                tokenized_examples["start_positions"].append(0)
                tokenized_examples["end_positions"].append(0)
            else: # the answer is in the span 
            # Move the token_start_index and token_end_index to the two ends of the answer.
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char_index: 
                    token_start_index += 1 # keep shifting to the right until token_start_index refers to where the answer starts in text 
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char_index: # start on right side and work down
                    token_end_index -= 1 # keep shifting left until token_end_index points to where answer ends in text 
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


In [10]:
data_collator = default_data_collator
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run8/checkpoint-20395'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

    

  0%|          | 0/131 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [11]:

args = TrainingArguments(
    f"drive/MyDrive/electra_eval_results",
    #evaluation_strategy = "steps",
    #max_steps=2,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy= 'epoch',
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    #compute_metrics= metrics,
)

Before we can evaluate the performance of the model, we need to map the predictions to the corresponding text in the context. 

The model output would be the loss,start logits, and end logits. 

reference: https://huggingface.co/transformers/main_classes/output.html#questionansweringmodeloutput

To see how we can make that into the required format for evaluation lets explore further with a batch from eval set. 

In [12]:
for batch in trainer.get_eval_dataloader():
    break
with torch.no_grad(): # inference mode 
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [9]:
output.start_logits.shape, output.end_logits.shape


(torch.Size([8, 512]), torch.Size([8, 512]))

batch size = 8 
max length = 512 

The most straight forward  way to go from logits to prediction text would be to just take index for the token with the highest logit in start and end logits. This would give us a value for the start and end index. 

This approach appears straight forward but could provide unintended results. For example, the end index could be less then start index or the model could predict that the answer is in the question. Without performing the checks there is no way to know if the predictions will be logically resonable. 

Alternatively, we can widen our search and validate that the predictions make sense before picking a final prediction for evalutation. 

To widen our search, we can take the top `n` start and end logits.
Using the top `n` indicies for start and end logits we can get all  possible combinations leading to a number of possible predictions. 
Before trying to see which combinations might be better then others we can validate that the combination leads to a valid result (within context and end index > start index). 

Out of all possible predictions with top `n` start and end logits the final prediction will be the one with the highest combined logit.   

In [6]:
n_best_size =10 # number of top values to consider for start/end logits 

In [18]:
# get logits for one example out of the batch  
start_logits = output.start_logits[0]
end_logits = output.end_logits[0]
start_indexes = torch.argsort(start_logits).tolist() # switch to last 10 if need other order 
end_indexes = torch.argsort(end_logits).tolist() # 

valid_answers = []
# try all possible combinations of start and end index based on logits 
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index:
          valid_answers.append({'score':start_logits[start_index] + end_logits[end_index],'text':''}) # need to get back to original text 


In [19]:
# check progress 
valid_answers[:30]

[{'score': tensor(-33.5883, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5808, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5771, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5735, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5650, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5643, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5594, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5540, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5508, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5505, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5491, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5446, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5428, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5407, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5386, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5379, device='cuda:0'), 'text': ''},
 {'score': tensor(-33.5302, device='cuda:0'), 'text': ''

- We can now sort valid answers by score and choose the best one to compute the evaluation metrics. 

- check if the possible answers are valid. 
- check if span given is part of context or question 

- each example could have multiple features (long context) so we need a way to map from feature back to example 
  - need unique identifier for each example in dataset 
  - need offset mapping to go from token indicies to character positions in the context. 


In [12]:
def prepare_validation_features(examples):
    # tokenize examples: truncate long contexts, keep overflows using stride, add padding,    
    tokenized_examples = tokenizer(
        examples["question" ],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # get mapping from features to corresponding example 
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []
    # iterate through all tokenized examples 
    for i in range(len(tokenized_examples["input_ids"])):
        # get sequence_ids corresponding to  example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 # since on the right tokenizer gives value of 1 

        # get index of example that this span is a part of 
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None) 
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [13]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

  0%|          | 0/12 [00:00<?, ?ba/s]

Now we can grab the predictions for all features by using the Trainer.predict method:

In [22]:
raw_predictions = trainer.predict(validation_features,re)


The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


In [32]:
raw_predictions

PredictionOutput(predictions=(array([[-12.823557 , -13.412564 , -13.089427 , ..., -15.701638 ,
        -15.508965 , -14.735753 ],
       [-13.580331 , -15.039974 , -15.820932 , ..., -16.112225 ,
        -16.04954  , -15.0015   ],
       [-12.700813 , -14.734307 , -15.1164055, ..., -16.276356 ,
        -16.186962 , -15.154984 ],
       ...,
       [ -8.663972 , -15.968044 , -15.435634 , ..., -16.310724 ,
        -16.087074 , -14.527416 ],
       [ 15.627633 , -15.600388 , -15.833546 , ..., -16.501022 ,
        -16.37523  , -15.411163 ],
       [ 14.555437 , -15.818826 , -15.6256485, ..., -16.401558 ,
        -16.248331 , -15.05848  ]], dtype=float32), array([[-13.331935, -15.225232, -14.260338, ..., -16.99073 , -16.78992 ,
        -16.732582],
       [-13.368929, -15.900547, -17.133322, ..., -18.163368, -18.119463,
        -18.099842],
       [-12.856957, -16.869467, -16.119864, ..., -17.673792, -17.585262,
        -17.70085 ],
       ...,
       [ -8.888439, -17.185217, -17.337   , ...


The Trainer hides the columns that are not used by the model.  (example_id and offset_mapping which we will need for our post-processing), so we set them back:

In [14]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

Since we set None in the offset mappings when it corresponds to a part of the question, it's easy to check if an answer is fully inside the context. 

We can also set a limit for the length of the answer to return. 

In [15]:
max_answer_length = 30


In [26]:
start_logits = output.start_logits[0]
end_logits = output.end_logits[0]
offset_mapping = validation_features[0]["offset_mapping"]

context = datasets["validation"][0]["context"]

# get the indices of the best start/end logits:
start_indexes = torch.argsort(start_logits)[-n_best_size:].tolist()
end_indexes = torch.argsort(end_logits)[-n_best_size:].tolist()
valid_answers = []
# check all combinations 
for start_index in start_indexes:
    for end_index in end_indexes:
        #ignore non-valid answers 
        if (
            start_index >= len(offset_mapping) # start of answer not in span 
            or end_index >= len(offset_mapping) # end_index not in span 
            or offset_mapping[start_index] is None # start index not in context but in question 
            or offset_mapping[end_index] is None # end index not in context but in question
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        # for valid answers in context 
        if start_index <= end_index: 
            start_char_index = offset_mapping[start_index][0]  # get index of start character in context 
            end_char_index = offset_mapping[end_index][1]   # get index of end character in context 
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char_index: end_char_index]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': tensor(29.8085, device='cuda:0'), 'text': 'France'},
 {'score': tensor(24.3578, device='cuda:0'), 'text': 'France.'},
 {'score': tensor(17.8913, device='cuda:0'),
  'text': 'Normandy, a region in France'},
 {'score': tensor(12.4405, device='cuda:0'),
  'text': 'Normandy, a region in France.'},
 {'score': tensor(9.6661, device='cuda:0'), 'text': 'in France'},
 {'score': tensor(8.7326, device='cuda:0'), 'text': 'region in France'},
 {'score': tensor(7.0556, device='cuda:0'),
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway'},
 {'score': tensor(6.9780, device='cuda:0'), 'text': 'a region in France'},
 {'score': tensor(5.7749, device='cuda:0'),
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland'},
 {'score': tensor(5.2743, device='cuda:0'), 'text': '.'}]

We can compare our results to the ground truth in the dataset 


In [27]:
datasets["validation"][0]["answers"]
print()

{'answer_start': [159, 159, 159, 159],
 'text': ['France', 'France', 'France', 'France']}



- Examples longer then the max length allowed by the model will have multiple features. To avoid making multipel predictions for the same example we need to get all answers for all the features generated by a given example and select the best one out of all of them 

- The example ID provides a unique identifier for each example in the dataset so we can use that to keep track of examples 

In [16]:
# map features to example IDs 
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)



- We need a way to deal with the situation when the question is not answerable. 

- If an example has multiple features then it is possible for one of them to have no answer simply because the answer is not part of the span of that text. 

- Therefore, an example should only return no answer if the best answer in each feature obtained from it is the impossible answer 



In [17]:
from tqdm.auto import tqdm

def postprocess_predictions(examples, features, raw_predictions, n_best_size = 10, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # instantiate ordered dict to store predictions 
    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # iterate through all examples 
    for example_index, example in enumerate(tqdm(examples)):
        # indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = 0 # score of no answer 
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # get predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # map positions in our logits to span of texts in the original context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = torch.argsort(torch.from_numpy(start_logits)).tolist()
            end_indexes = torch.argsort(torch.from_numpy(end_logits)).tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # ignore non-valid answers 
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        # if we have valid answer best answer is one with highest score
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # if no valid answers 
            best_answer = {"text": "", "score": 0.0}
        # get best answer and add to predictions 
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

And we can apply our post-processing function to our raw predictions:



In [36]:
final_predictions = postprocess_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

The datasets library provides a handy function to help compute the metrics associated with the SQuAD dataset. 

In [18]:
metric = load_metric("squad_v2")

Downloading:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

The metric function expects a list of dictionaries so we need to format our predictions before being able to compute the metrics. 

In [39]:
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'HasAns_exact': 67.44264507422402,
 'HasAns_f1': 75.02581564144072,
 'HasAns_total': 5928,
 'NoAns_exact': 69.25147182506308,
 'NoAns_f1': 69.25147182506308,
 'NoAns_total': 5945,
 'best_exact': 68.35677587804261,
 'best_exact_thresh': 0.0,
 'best_f1': 72.14293229364637,
 'best_f1_thresh': 0.0,
 'exact': 68.34835340688959,
 'f1': 72.13450982249314,
 'total': 11873}

# Function for eval model 

Now that we now how to get metrics for a single example we can put this all together for use in a function, 

In [19]:
from tqdm.auto import tqdm
import torch
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator
from datasets import load_dataset, load_metric 
from transformers import AutoTokenizer
import numpy as np

max_length=512 
doc_stride= 128
max_answer_length = 30
n_best_size =10 
model_checkpoint= '' # enter path to checkpoint here 

In [20]:
def checkpoint_eval(model_checkpoint):
  datasets = load_dataset("squad_v2")

  data_collator = default_data_collator
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

  args = TrainingArguments(
    f"drive/MyDrive/electra_eval_results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy= 'epoch',
  )

  trainer = Trainer(
      model,
      args,
      train_dataset=tokenized_datasets["train"],
      eval_dataset=tokenized_datasets["validation"],
      data_collator=data_collator,
      tokenizer=tokenizer,
  )

  def prepare_validation_features(examples):
    # tokenize examples
    tokenized_examples = tokenizer(
        examples["question" ],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # mapping from features to corresponding example 
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []
    # iterate through all tokenized examples 
    for i in range(len(tokenized_examples["input_ids"])):
        # get sequence_ids corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 # since on the right tokenizer gives value of 1 

        # get index of example containing current feature 
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # use None to indicate that the tokenis not part of context 
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None) 
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples
# get validation features 
  validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)
# get raw predictions of logits 
  raw_predictions = trainer.predict(validation_features)
  validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))  

    # map features to example IDS 
  import collections

  examples = datasets["validation"]
  features = validation_features

  example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
  features_per_example = collections.defaultdict(list)
  for i, feature in enumerate(features):
      features_per_example[example_id_to_index[feature["example_id"]]].append(i)

# post process raw predictions for use with metrics function 
  def postprocess_predictions(examples, features, raw_predictions, n_best_size = 10, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)


    predictions = collections.OrderedDict()


    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # iterate through examples
    for example_index, example in enumerate(tqdm(examples)):
        # get indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = 0 # score of impossible answer 
        valid_answers = []
        
        context = example["context"]
        # iterate through all features for current example 
        for feature_index in feature_indices:
            # get predictions for current feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # map positions in logits to span of texts in original context 
            offset_mapping = features[feature_index]["offset_mapping"]

            # update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = torch.argsort(torch.from_numpy(start_logits)).tolist()
            end_indexes = torch.argsort(torch.from_numpy(end_logits)).tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:

                    # ignore non-valid questions 
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0: # if we have any valid answers 
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else: # if no valid answers 
            best_answer = {"text": "", "score": 0.0}
            # get best answer and store prediction for best answer 
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions
  # post precess predictions 
  final_predictions = postprocess_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

  metric = load_metric("squad_v2")
  # format predictions for use with metric function 
  formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
  references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
  return metric.compute(predictions=formatted_predictions, references=references)








# Run 1 Epoch 1



In [42]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e1' # after first epoch 


In [46]:
checkpoint_eval(model_checkpoint) 

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e1/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e1/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e1/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e1/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e1/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e1/config.json
Model config ElectraConfig {
  "_name_or_path": "google/electra-small-discriminator",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra"

Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 60.121457489878544,
 'HasAns_f1': 67.37903533556687,
 'HasAns_total': 5928,
 'NoAns_exact': 57.20773759461733,
 'NoAns_f1': 57.20773759461733,
 'NoAns_total': 5945,
 'best_exact': 58.68777899435695,
 'best_exact_thresh': 0.0,
 'best_f1': 62.30294967314454,
 'best_f1_thresh': 0.0,
 'exact': 58.662511580897835,
 'f1': 62.28610473083832,
 'total': 11873}

## e2 

checkpoint-8158 



In [47]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e2'
checkpoint_eval(model_checkpoint) 

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e2/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e2/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e2/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e2/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e2/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e2/config.json
Model config ElectraConfig {
  "_name_or_path": "google/electra-small-discriminator",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra"

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 60.07085020242915,
 'HasAns_f1': 66.27789852010258,
 'HasAns_total': 5928,
 'NoAns_exact': 71.01766190075693,
 'NoAns_f1': 71.01766190075693,
 'NoAns_total': 5945,
 'best_exact': 65.55209298408153,
 'best_exact_thresh': 0.0,
 'best_f1': 68.65117345465953,
 'best_f1_thresh': 0.0,
 'exact': 65.55209298408153,
 'f1': 68.65117345465941,
 'total': 11873}

## epoch 3 
checkpoint-12237


In [48]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e3' # after first epoch 
checkpoint_eval(model_checkpoint) 

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e3/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e3/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e3/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e3/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e3/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e3/config.json
Model config ElectraConfig {
  "_name_or_path": "google/electra-small-discriminator",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra"

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 67.76315789473684,
 'HasAns_f1': 74.72986741583328,
 'HasAns_total': 5928,
 'NoAns_exact': 60.30277544154752,
 'NoAns_f1': 60.30277544154752,
 'NoAns_total': 5945,
 'best_exact': 64.03604817653499,
 'best_exact_thresh': 0.0,
 'best_f1': 67.5144153997359,
 'best_f1_thresh': 0.0,
 'exact': 64.02762570538196,
 'f1': 67.50599292858273,
 'total': 11873}

## epoch 4 

checkpoint-16316

In [53]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e4' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e4/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e4/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e4/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e4/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e4/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e4/config.json
Model config ElectraConfig {
  "_name_or_path": "google/electra-small-discriminator",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra"

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 64.389338731444,
 'HasAns_f1': 70.56546947321925,
 'HasAns_total': 5928,
 'NoAns_exact': 69.70563498738436,
 'NoAns_f1': 69.70563498738436,
 'NoAns_total': 5945,
 'best_exact': 67.05971532047502,
 'best_exact_thresh': 0.0,
 'best_f1': 70.14335913730709,
 'best_f1_thresh': 0.0,
 'exact': 67.051292849322,
 'f1': 70.13493666615399,
 'total': 11873}

## Epoch 5 

checkpoint-20395

In [54]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e5' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e5/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e5/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e5/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e5/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e5/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run1-checkpoint-e5/config.json
Model config ElectraConfig {
  "_name_or_path": "google/electra-small-discriminator",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra"

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 67.27395411605939,
 'HasAns_f1': 73.75078684508728,
 'HasAns_total': 5928,
 'NoAns_exact': 65.58452481076534,
 'NoAns_f1': 65.58452481076534,
 'NoAns_total': 5945,
 'best_exact': 66.43645245515035,
 'best_exact_thresh': 0.0,
 'best_f1': 69.67023199003464,
 'best_f1_thresh': 0.0,
 'exact': 66.4280299839973,
 'f1': 69.66180951888148,
 'total': 11873}

# Run 2 

## Epoch 6 

checkpoint-4079

In [60]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e6'
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e6/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e6/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e6/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e6/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e6/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e6/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra_5e.pt",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "nu

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 68.85964912280701,
 'HasAns_f1': 75.6480392933267,
 'HasAns_total': 5928,
 'NoAns_exact': 62.80908326324643,
 'NoAns_f1': 62.80908326324643,
 'NoAns_total': 5945,
 'best_exact': 65.83845700328476,
 'best_exact_thresh': 0.0,
 'best_f1': 69.22779221181209,
 'best_f1_thresh': 0.0,
 'exact': 65.83003453213173,
 'f1': 69.21936974065889,
 'total': 11873}

## epoch 7

checkpoint-8158

In [63]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e7' 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e7/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e7/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e7/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e7/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e7/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e7/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra_5e.pt",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "nu

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 66.26180836707152,
 'HasAns_f1': 72.73302408747601,
 'HasAns_total': 5928,
 'NoAns_exact': 70.56349873843565,
 'NoAns_f1': 70.56349873843565,
 'NoAns_total': 5945,
 'best_exact': 68.42415564726691,
 'best_exact_thresh': 0.0,
 'best_f1': 71.65513069911226,
 'best_f1_thresh': 0.0,
 'exact': 68.41573317611387,
 'f1': 71.64670822795924,
 'total': 11873}

## Epoch 8 

checkpoint-12237

In [64]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e8' 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e8/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e8/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e8/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e8/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e8/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e8/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra_5e.pt",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "nu

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 70.3778677462888,
 'HasAns_f1': 77.5506935974699,
 'HasAns_total': 5928,
 'NoAns_exact': 60.94196804037006,
 'NoAns_f1': 60.94196804037006,
 'NoAns_total': 5945,
 'best_exact': 65.661585109071,
 'best_exact_thresh': 0.0,
 'best_f1': 69.24286293656246,
 'best_f1_thresh': 0.0,
 'exact': 65.65316263791796,
 'f1': 69.23444046540924,
 'total': 11873}

# E9 

checkpoint-16316

In [65]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e9' 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e9/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e9/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e9/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e9/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e9/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e9/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra_5e.pt",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "nu

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 66.63292847503374,
 'HasAns_f1': 73.03379212699842,
 'HasAns_total': 5928,
 'NoAns_exact': 70.71488645920942,
 'NoAns_f1': 70.71488645920942,
 'NoAns_total': 5945,
 'best_exact': 68.68525225301103,
 'best_exact_thresh': 0.0,
 'best_f1': 71.88110163638918,
 'best_f1_thresh': 0.0,
 'exact': 68.676829781858,
 'f1': 71.87267916523612,
 'total': 11873}

## E10 
checkpoint-20395

In [66]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e10' 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e10/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e10/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e10/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e10/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e10/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e10/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra_5e.pt",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 64.64237516869096,
 'HasAns_f1': 70.49038402570005,
 'HasAns_total': 5928,
 'NoAns_exact': 74.70142977291842,
 'NoAns_f1': 74.70142977291842,
 'NoAns_total': 5945,
 'best_exact': 69.68752632022235,
 'best_exact_thresh': 0.0,
 'best_f1': 72.60734410042554,
 'best_f1_thresh': 0.0,
 'exact': 69.67910384906932,
 'f1': 72.59892162927244,
 'total': 11873}

## E 11 
checkpoint-24474 

In [67]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e11' 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e11/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e11/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e11/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e11/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e11/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e11/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra_5e.pt",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 65.48582995951416,
 'HasAns_f1': 71.57815577404901,
 'HasAns_total': 5928,
 'NoAns_exact': 74.19680403700589,
 'NoAns_f1': 74.19680403700589,
 'NoAns_total': 5945,
 'best_exact': 69.85597574328308,
 'best_exact_thresh': 0.0,
 'best_f1': 72.89777709328423,
 'best_f1_thresh': 0.0,
 'exact': 69.84755327213004,
 'f1': 72.8893546221312,
 'total': 11873}

## E 12 
checkpoint-28553 

In [68]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e12' 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e12/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e12/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e12/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e12/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e12/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e12/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra_5e.pt",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 66.59919028340082,
 'HasAns_f1': 73.12432025396403,
 'HasAns_total': 5928,
 'NoAns_exact': 71.75777964676199,
 'NoAns_f1': 71.75777964676199,
 'NoAns_total': 5945,
 'best_exact': 69.19060052219321,
 'best_exact_thresh': 0.0,
 'best_f1': 72.44849410136452,
 'best_f1_thresh': 0.0,
 'exact': 69.18217805104018,
 'f1': 72.44007163021149,
 'total': 11873}

## e 13 

checkpoint-32632


In [69]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e13' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e13/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e13/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e13/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e13/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e13/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e13/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra_5e.pt",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 70.27665317139001,
 'HasAns_f1': 77.38542592262827,
 'HasAns_total': 5928,
 'NoAns_exact': 65.29857022708158,
 'NoAns_f1': 65.29857022708158,
 'NoAns_total': 5945,
 'best_exact': 67.79247031078918,
 'best_exact_thresh': 0.0,
 'best_f1': 71.34176744456681,
 'best_f1_thresh': 0.0,
 'exact': 67.78404783963614,
 'f1': 71.33334497341372,
 'total': 11873}

## e14 

checkpoint-36711

In [70]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e14' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e14/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e14/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e14/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e14/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e14/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e14/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra_5e.pt",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 69.97300944669365,
 'HasAns_f1': 76.9116390678931,
 'HasAns_total': 5928,
 'NoAns_exact': 66.54331370899916,
 'NoAns_f1': 66.54331370899916,
 'NoAns_total': 5945,
 'best_exact': 68.26412869535922,
 'best_exact_thresh': 0.0,
 'best_f1': 71.72847607129387,
 'best_f1_thresh': 0.0,
 'exact': 68.25570622420618,
 'f1': 71.72005360014082,
 'total': 11873}

## e15 
checkpoint-40790

In [71]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e15' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e15/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e15/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e15/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e15/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e15/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e15/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra_5e.pt",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 66.63292847503374,
 'HasAns_f1': 73.210726807293,
 'HasAns_total': 5928,
 'NoAns_exact': 71.75777964676199,
 'NoAns_f1': 71.75777964676199,
 'NoAns_total': 5945,
 'best_exact': 69.20744546449929,
 'best_exact_thresh': 0.0,
 'best_f1': 72.49163551870919,
 'best_f1_thresh': 0.0,
 'exact': 69.19902299334625,
 'f1': 72.48321304755613,
 'total': 11873}

## e16 
checkpoint-44869


In [72]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e16' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e16/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e16/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e16/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e16/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e16/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e16/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra_5e.pt",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 68.69095816464238,
 'HasAns_f1': 76.37793655992425,
 'HasAns_total': 5928,
 'NoAns_exact': 67.35071488645922,
 'NoAns_f1': 67.35071488645922,
 'NoAns_total': 5945,
 'best_exact': 68.02829950307421,
 'best_exact_thresh': 0.0,
 'best_f1': 71.86628551564341,
 'best_f1_thresh': 0.0,
 'exact': 68.01987703192117,
 'f1': 71.85786304449024,
 'total': 11873}

## e17 
checkpoint-48948

In [73]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e17' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e17/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e17/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e17/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e17/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e17/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e17/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra_5e.pt",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 68.20175438596492,
 'HasAns_f1': 75.2854986054221,
 'HasAns_total': 5928,
 'NoAns_exact': 67.82169890664424,
 'NoAns_f1': 67.82169890664424,
 'NoAns_total': 5945,
 'best_exact': 68.01987703192117,
 'best_exact_thresh': 0.0,
 'best_f1': 71.55667781798576,
 'best_f1_thresh': 0.0,
 'exact': 68.01145456076813,
 'f1': 71.54825534683268,
 'total': 11873}

## e18

checkpoint-57106

In [74]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e18' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e18/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e18/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e18/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e18/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e18/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run2_checkpoint-e18/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra_5e.pt",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 69.0620782726046,
 'HasAns_f1': 76.70178847874728,
 'HasAns_total': 5928,
 'NoAns_exact': 66.20689655172414,
 'NoAns_f1': 66.20689655172414,
 'NoAns_total': 5945,
 'best_exact': 67.64086583003453,
 'best_exact_thresh': 0.0,
 'best_f1': 71.45525158780563,
 'best_f1_thresh': 0.0,
 'exact': 67.6324433588815,
 'f1': 71.44682911665251,
 'total': 11873}

# Run 3 

## e19

Run 3 
drive/MyDrive/electra-small-squad_v2_run3/checkpoint-4079 

In [ ]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run3/checkpoint-4079' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 69.2476383265857,
 'HasAns_f1': 77.03798290596976,
 'HasAns_total': 5928,
 'NoAns_exact': 66.17325483599663,
 'NoAns_f1': 66.17325483599663,
 'NoAns_total': 5945,
 'best_exact': 67.70824559925882,
 'best_exact_thresh': 0.0,
 'best_f1': 71.5978407029892,
 'best_f1_thresh': 0.0,
 'exact': 67.70824559925882,
 'f1': 71.59784070298909,
 'total': 11873}

## e20 

drive/MyDrive/electra-small-squad_v2_run3/checkpoint-8158

In [ ]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run3/checkpoint-8158' 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-8158/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-8158/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-8158/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-8158/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-8158/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-8158/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/checkpoint-57106",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 67.88124156545209,
 'HasAns_f1': 76.02518607102255,
 'HasAns_total': 5928,
 'NoAns_exact': 67.08158116063919,
 'NoAns_f1': 67.08158116063919,
 'NoAns_total': 5945,
 'best_exact': 67.48926134927987,
 'best_exact_thresh': 0.0,
 'best_f1': 71.5554032703633,
 'best_f1_thresh': 0.0,
 'exact': 67.48083887812685,
 'f1': 71.54698079921019,
 'total': 11873}

##  epoch 21 

drive/MyDrive/electra-small-squad_v2_run3/checkpoint-12237

In [75]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run3/checkpoint-12237' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-12237/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-12237/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-12237/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-12237/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-12237/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-12237/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/checkpoint-57106",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_ty

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 68.4885290148448,
 'HasAns_f1': 76.3549522204533,
 'HasAns_total': 5928,
 'NoAns_exact': 67.65349032800673,
 'NoAns_f1': 67.65349032800673,
 'NoAns_total': 5945,
 'best_exact': 68.07883432999242,
 'best_exact_thresh': 0.0,
 'best_f1': 72.00641428138204,
 'best_f1_thresh': 0.0,
 'exact': 68.07041185883938,
 'f1': 71.99799181022891,
 'total': 11873}

## e 22 
drive/MyDrive/electra-small-squad_v2_run3/checkpoint-16316

In [76]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run3/checkpoint-16316' 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-16316/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-16316/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-16316/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-16316/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-16316/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-16316/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/checkpoint-57106",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_ty

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 67.03778677462888,
 'HasAns_f1': 74.42162108977959,
 'HasAns_total': 5928,
 'NoAns_exact': 71.35407905803196,
 'NoAns_f1': 71.35407905803196,
 'NoAns_total': 5945,
 'best_exact': 69.20744546449929,
 'best_exact_thresh': 0.0,
 'best_f1': 72.89407646089565,
 'best_f1_thresh': 0.0,
 'exact': 69.19902299334625,
 'f1': 72.88565398974255,
 'total': 11873}

## e 23 
drive/MyDrive/electra-small-squad_v2_run3/checkpoint-20395

In [77]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run3/checkpoint-20395' 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-20395/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-20395/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-20395/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-20395/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-20395/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-20395/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/checkpoint-57106",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_ty

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 68.42105263157895,
 'HasAns_f1': 75.46005592841817,
 'HasAns_total': 5928,
 'NoAns_exact': 68.7636669470143,
 'NoAns_f1': 68.7636669470143,
 'NoAns_total': 5945,
 'best_exact': 68.60102754148068,
 'best_exact_thresh': 0.0,
 'best_f1': 72.11548989671226,
 'best_f1_thresh': 0.0,
 'exact': 68.59260507032764,
 'f1': 72.10706742555915,
 'total': 11873}

## e 24 
drive/MyDrive/electra-small-squad_v2_run3/checkpoint-24474

In [ ]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run3/checkpoint-24474' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-24474/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-24474/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-24474/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-24474/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-24474/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-24474/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/checkpoint-57106",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_ty

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 68.91025641025641,
 'HasAns_f1': 76.57791473445008,
 'HasAns_total': 5928,
 'NoAns_exact': 67.1825063078217,
 'NoAns_f1': 67.1825063078217,
 'NoAns_total': 5945,
 'best_exact': 68.05356691653331,
 'best_exact_thresh': 0.0,
 'best_f1': 71.88190672499137,
 'best_f1_thresh': 0.0,
 'exact': 68.04514444538027,
 'f1': 71.87348425383824,
 'total': 11873}

## e25

drive/MyDrive/electra-small-squad_v2_run3/checkpoint-28553

In [ ]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run3/checkpoint-28553' 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-28553/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-28553/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-28553/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-28553/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-28553/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-28553/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/checkpoint-57106",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_ty

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 66.78475033738192,
 'HasAns_f1': 74.14686781182013,
 'HasAns_total': 5928,
 'NoAns_exact': 71.2363330529857,
 'NoAns_f1': 71.2363330529857,
 'NoAns_total': 5945,
 'best_exact': 69.02215109913249,
 'best_exact_thresh': 0.0,
 'best_f1': 72.69793922247726,
 'best_f1_thresh': 0.0,
 'exact': 69.01372862797945,
 'f1': 72.68951675132409,
 'total': 11873}

## e26 

drive/MyDrive/electra-small-squad_v2_run3/checkpoint-32632

In [ ]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run3/checkpoint-32632' 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-32632/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-32632/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-32632/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-32632/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-32632/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-32632/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/checkpoint-57106",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_ty

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 67.98245614035088,
 'HasAns_f1': 75.5741136834711,
 'HasAns_total': 5928,
 'NoAns_exact': 69.57106812447435,
 'NoAns_f1': 69.57106812447435,
 'NoAns_total': 5945,
 'best_exact': 68.78632190684746,
 'best_exact_thresh': 0.0,
 'best_f1': 72.57671573449159,
 'best_f1_thresh': 0.0,
 'exact': 68.77789943569444,
 'f1': 72.5682932633385,
 'total': 11873}

## e27 

drive/MyDrive/electra-small-squad_v2_run3/checkpoint-36711

In [ ]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run3/checkpoint-36711' 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-36711/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-36711/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-36711/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-36711/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-36711/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-36711/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/checkpoint-57106",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_ty

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 67.39203778677462,
 'HasAns_f1': 74.53152981803619,
 'HasAns_total': 5928,
 'NoAns_exact': 70.3784693019344,
 'NoAns_f1': 70.3784693019344,
 'NoAns_total': 5945,
 'best_exact': 68.89581403183693,
 'best_exact_thresh': 0.0,
 'best_f1': 72.46044881338507,
 'best_f1_thresh': 0.0,
 'exact': 68.88739156068391,
 'f1': 72.45202634223193,
 'total': 11873}

## e28 

drive/MyDrive/electra-small-squad_v2_run3/checkpoint-40790



In [78]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run3/checkpoint-40790' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-40790/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-40790/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-40790/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-40790/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-40790/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run3/checkpoint-40790/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/checkpoint-57106",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_ty

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 68.16801619433198,
 'HasAns_f1': 75.61100052461717,
 'HasAns_total': 5928,
 'NoAns_exact': 68.81412952060555,
 'NoAns_f1': 68.81412952060555,
 'NoAns_total': 5945,
 'best_exact': 68.49995788764423,
 'best_exact_thresh': 0.0,
 'best_f1': 72.21612154551782,
 'best_f1_thresh': 0.0,
 'exact': 68.4915354164912,
 'f1': 72.20769907436465,
 'total': 11873}

# Run 4 

## e29 
drive/MyDrive/electra-small-squad_v2_run4/checkpoint-4079




In [79]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run4/checkpoint-4079' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-4079/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-4079/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-4079/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-4079/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-4079/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-4079/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/run3-checkpoint-40790",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_typ

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 64.60863697705803,
 'HasAns_f1': 71.7850994144206,
 'HasAns_total': 5928,
 'NoAns_exact': 74.4322960470984,
 'NoAns_f1': 74.4322960470984,
 'NoAns_total': 5945,
 'best_exact': 69.5359218394677,
 'best_exact_thresh': 0.0,
 'best_f1': 73.11901535658102,
 'best_f1_thresh': 0.0,
 'exact': 69.52749936831466,
 'f1': 73.11059288542798,
 'total': 11873}

## e 30 

drive/MyDrive/electra-small-squad_v2_run4/checkpoint-8158

In [ ]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run4/checkpoint-8158' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-8158/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-8158/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-8158/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-8158/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-8158/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-8158/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/run3-checkpoint-40790",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_typ

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 66.76788124156545,
 'HasAns_f1': 75.64957723752339,
 'HasAns_total': 5928,
 'NoAns_exact': 67.70395290159799,
 'NoAns_f1': 67.70395290159799,
 'NoAns_total': 5945,
 'best_exact': 67.24500968584182,
 'best_exact_thresh': 0.0,
 'best_f1': 71.67949918841421,
 'best_f1_thresh': 0.0,
 'exact': 67.2365872146888,
 'f1': 71.67107671726104,
 'total': 11873}

## e 31 
drive/MyDrive/electra-small-squad_v2_run4/checkpoint-12237

In [ ]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run4/checkpoint-12237' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-12237/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-12237/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-12237/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-12237/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-12237/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-12237/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/run3-checkpoint-40790",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "mod

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 64.82793522267207,
 'HasAns_f1': 72.70575532776756,
 'HasAns_total': 5928,
 'NoAns_exact': 73.10344827586206,
 'NoAns_f1': 73.10344827586206,
 'NoAns_total': 5945,
 'best_exact': 68.9800387433673,
 'best_exact_thresh': 0.0,
 'best_f1': 72.91330898534561,
 'best_f1_thresh': 0.0,
 'exact': 68.97161627221426,
 'f1': 72.90488651419246,
 'total': 11873}

## e 32 

drive/MyDrive/electra-small-squad_v2_run4/checkpoint-16316

In [80]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run4/checkpoint-16316' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-16316/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-16316/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-16316/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-16316/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-16316/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-16316/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/run3-checkpoint-40790",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "mod

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 64.76045883940621,
 'HasAns_f1': 72.00756560558041,
 'HasAns_total': 5928,
 'NoAns_exact': 74.31455004205215,
 'NoAns_f1': 74.31455004205215,
 'NoAns_total': 5945,
 'best_exact': 69.55276678177377,
 'best_exact_thresh': 0.0,
 'best_f1': 73.17113188830817,
 'best_f1_thresh': 0.0,
 'exact': 69.54434431062073,
 'f1': 73.16270941715511,
 'total': 11873}

## e 33 

drive/MyDrive/electra-small-squad_v2_run4/checkpoint-20395


In [ ]:
model_checkpoint= 'drive/MyDrive/electra-small-squad_v2_run4/checkpoint-20395' # after first epoch 
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-20395/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-20395/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-20395/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-20395/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-20395/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-20395/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/run3-checkpoint-40790",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "mod

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 65.90755735492577,
 'HasAns_f1': 73.1168899049929,
 'HasAns_total': 5928,
 'NoAns_exact': 71.60639192598822,
 'NoAns_f1': 71.60639192598822,
 'NoAns_total': 5945,
 'best_exact': 68.7694769645414,
 'best_exact_thresh': 0.0,
 'best_f1': 72.36898200596313,
 'best_f1_thresh': 0.0,
 'exact': 68.76105449338836,
 'f1': 72.36055953481001,
 'total': 11873}

## E 34 



In [21]:
model_checkpoint = 'drive/MyDrive/electra-small-squad_v2_run4/checkpoint-24474'
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-24474/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-24474/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-24474/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-24474/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-24474/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-24474/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/run3-checkpoint-40790",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "mod

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 66.34615384615384,
 'HasAns_f1': 73.73904292168108,
 'HasAns_total': 5928,
 'NoAns_exact': 70.81581160639193,
 'NoAns_f1': 70.81581160639193,
 'NoAns_total': 5945,
 'best_exact': 68.59260507032764,
 'best_exact_thresh': 0.0,
 'best_f1': 72.28375696451856,
 'best_f1_thresh': 0.0,
 'exact': 68.5841825991746,
 'f1': 72.27533449336539,
 'total': 11873}

## E 35 


In [22]:
model_checkpoint = 'drive/MyDrive/electra-small-squad_v2_run4/checkpoint-28553'
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-28553/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-28553/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-28553/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-28553/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-28553/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-28553/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/run3-checkpoint-40790",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "mod

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 66.8859649122807,
 'HasAns_f1': 74.15217255353556,
 'HasAns_total': 5928,
 'NoAns_exact': 70.05887300252313,
 'NoAns_f1': 70.05887300252313,
 'NoAns_total': 5945,
 'best_exact': 68.48311294533816,
 'best_exact_thresh': 0.0,
 'best_f1': 72.11101481490458,
 'best_f1_thresh': 0.0,
 'exact': 68.47469047418512,
 'f1': 72.10259234375143,
 'total': 11873}

## E 36 

drive/MyDrive/electra-small-squad_v2_run4/checkpoint-32632

In [23]:
model_checkpoint = 'drive/MyDrive/electra-small-squad_v2_run4/checkpoint-32632'
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-32632/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-32632/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-32632/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-32632/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-32632/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-32632/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/run3-checkpoint-40790",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "mod

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 67.51012145748987,
 'HasAns_f1': 75.25469222972556,
 'HasAns_total': 5928,
 'NoAns_exact': 69.03280067283431,
 'NoAns_f1': 69.03280067283431,
 'NoAns_total': 5945,
 'best_exact': 68.2809736376653,
 'best_exact_thresh': 0.0,
 'best_f1': 72.14771460774998,
 'best_f1_thresh': 0.0,
 'exact': 68.27255116651226,
 'f1': 72.13929213659685,
 'total': 11873}

## E  37 
drive/MyDrive/electra-small-squad_v2_run4/checkpoint-36711


In [24]:
model_checkpoint = 'drive/MyDrive/electra-small-squad_v2_run4/checkpoint-36711'
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-36711/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-36711/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-36711/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-36711/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-36711/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-36711/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/run3-checkpoint-40790",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "mod

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 66.0425101214575,
 'HasAns_f1': 73.19091347814346,
 'HasAns_total': 5928,
 'NoAns_exact': 71.94280908326324,
 'NoAns_f1': 71.94280908326324,
 'NoAns_total': 5945,
 'best_exact': 69.0053061568264,
 'best_exact_thresh': 0.0,
 'best_f1': 72.5743902213793,
 'best_f1_thresh': 0.0,
 'exact': 68.99688368567338,
 'f1': 72.56596775022619,
 'total': 11873}

## E 38
drive/MyDrive/electra-small-squad_v2_run4/checkpoint-40790

In [26]:
model_checkpoint = 'drive/MyDrive/electra-small-squad_v2_run4/checkpoint-40790'
checkpoint_eval(model_checkpoint)

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-40790/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-40790/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-40790/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-40790/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-40790/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run4/checkpoint-40790/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/run3-checkpoint-40790",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "mod

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'HasAns_exact': 67.02091767881241,
 'HasAns_f1': 74.59140378260783,
 'HasAns_total': 5928,
 'NoAns_exact': 69.92430613961312,
 'NoAns_f1': 69.92430613961312,
 'NoAns_total': 5945,
 'best_exact': 68.48311294533816,
 'best_exact_thresh': 0.0,
 'best_f1': 72.26293621016607,
 'best_f1_thresh': 0.0,
 'exact': 68.47469047418512,
 'f1': 72.2545137390129,
 'total': 11873}

# Run 5 

## Epoch 39 



In [ ]:
base_path = 'drive/MyDrive/electra-small-squad_v2_run5/'
run_5_epochs = ['checkpoint-4079', 'checkpoint-8158','checkpoint-12237','checkpoint-16316','checkpoint-20395','checkpoint-24474','checkpoint-28553',
 'checkpoint-32632','checkpoint-36711','checkpoint-40790' ] # from saved checkpoints 
e = 39 # track epoch being evaluated 
metric_run5=[]
for checkpoint in run_5_epochs:
  print(f'EVAL EPOCH {e}')
  e +=1
  metric_run5.append(checkpoint_eval(base_path+checkpoint))
  print(f'{len(metric_run5)} Checkpoints done')
  print('\n')
  print('\n')
  


EVAL EPOCH 39


Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run5/checkpoint-4079/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run5/checkpoint-4079/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run5/checkpoint-4079/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run5/checkpoint-4079/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run5/checkpoint-4079/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run5/checkpoint-4079/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra-small-squad_v2_run4/checkpoint-40790",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddi

Post-processing 11873 example predictions split into 11969 features.


...

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

10 Checkpoints done






In [ ]:
metric_run5

[{'HasAns_exact': 64.06882591093117,
  'HasAns_f1': 71.07968826825265,
  'HasAns_total': 5928,
  'NoAns_exact': 75.1555929352397,
  'NoAns_f1': 75.1555929352397,
  'NoAns_total': 5945,
  'best_exact': 69.62014655099806,
  'best_exact_thresh': 0.0,
  'best_f1': 73.12055858285211,
  'best_f1_thresh': 0.0,
  'exact': 69.62014655099806,
  'f1': 73.1205585828521,
  'total': 11873},
 {'HasAns_exact': 64.0519568151147,
  'HasAns_f1': 71.83336968510022,
  'HasAns_total': 5928,
  'NoAns_exact': 72.91841883936081,
  'NoAns_f1': 72.91841883936081,
  'NoAns_total': 5945,
  'best_exact': 68.49995788764423,
  'best_exact_thresh': 0.0,
  'best_f1': 72.38509353097584,
  'best_f1_thresh': 0.0,
  'exact': 68.4915354164912,
  'f1': 72.37667105982277,
  'total': 11873},
 {'HasAns_exact': 62.7361673414305,
  'HasAns_f1': 69.70352484826316,
  'HasAns_total': 5928,
  'NoAns_exact': 76.080740117746,
  'NoAns_f1': 76.080740117746,
  'NoAns_total': 5945,
  'best_exact': 69.42642971447823,
  'best_exact_thresh':

# Run 6 

## Epoch 49 -58



In [27]:
base_path = 'drive/MyDrive/electra-small-squad_v2_run6/'
run_6_epochs = ['checkpoint-4079', 'checkpoint-8158','checkpoint-12237','checkpoint-16316','checkpoint-20395','checkpoint-24474','checkpoint-28553',
 'checkpoint-32632','checkpoint-36711','checkpoint-40790' ] # from saved checkpoints 
e = 49 # track epoch being evaluated 
metric_run6 =[]
for checkpoint in run_6_epochs:
  print(f'EVAL EPOCH {e} Checkpoint')
  e +=1
  metric_run6.append(checkpoint_eval(base_path+checkpoint))
  print('\n')
  print('\n')
  print(f'{len(metric_run6)} Checkpoints done')
  

EVAL EPOCH 49 Checkpoint


Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run6/checkpoint-4079/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run6/checkpoint-4079/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run6/checkpoint-4079/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run6/checkpoint-4079/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run6/checkpoint-4079/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run6/checkpoint-4079/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra-small-squad_v2_run5/checkpoint-40790",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddi

  0%|          | 0/12 [00:00<?, ?ba/s]

...

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]





10 Checkpoints done


In [28]:
metric_run6

[{'HasAns_exact': 67.89811066126856,
  'HasAns_f1': 75.94643844028683,
  'HasAns_total': 5928,
  'NoAns_exact': 65.33221194280908,
  'NoAns_f1': 65.33221194280908,
  'NoAns_total': 5945,
  'best_exact': 66.62174682051715,
  'best_exact_thresh': 0.0,
  'best_f1': 70.64014883129983,
  'best_f1_thresh': 0.0,
  'exact': 66.6133243493641,
  'f1': 70.6317263601467,
  'total': 11873},
 {'HasAns_exact': 67.03778677462888,
  'HasAns_f1': 75.52696997472496,
  'HasAns_total': 5928,
  'NoAns_exact': 65.88730025231287,
  'NoAns_f1': 65.88730025231287,
  'NoAns_total': 5945,
  'best_exact': 66.47014233976249,
  'best_exact_thresh': 0.0,
  'best_f1': 70.70865644825848,
  'best_f1_thresh': 0.0,
  'exact': 66.46171986860945,
  'f1': 70.70023397710528,
  'total': 11873},
 {'HasAns_exact': 67.15587044534414,
  'HasAns_f1': 74.97602785017777,
  'HasAns_total': 5928,
  'NoAns_exact': 68.37678721614802,
  'NoAns_f1': 68.37678721614802,
  'NoAns_total': 5945,
  'best_exact': 67.77562536848312,
  'best_exact_

# Run 7 



In [ ]:
base_path = 'drive/MyDrive/electra-small-squad_v2_run7/'
run_7_epochs = ['checkpoint-4079', 'checkpoint-8158','checkpoint-12237','checkpoint-16316','checkpoint-20395',] # from saved checkpoints 
e = 59 # track epoch being evaluated 
metric_run7=[]
for checkpoint in run_7_epochs:
  print(f'EVAL EPOCH {e}')
  e +=1
  metric_run7.append(checkpoint_eval(base_path+checkpoint))
  print(f'{len(metric_run7)} Checkpoints done')
  print('\n')
  print('\n')
  

EVAL EPOCH 59


Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run7/checkpoint-4079/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run7/checkpoint-4079/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run7/checkpoint-4079/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run7/checkpoint-4079/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run7/checkpoint-4079/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run7/checkpoint-4079/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra-small-squad_v2_run6/checkpoint-40790",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddi

  0%|          | 0/12 [00:00<?, ?ba/s]

...

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

5 Checkpoints done






In [ ]:
metric_run7

[{'HasAns_exact': 63.76518218623482,
  'HasAns_f1': 71.75756694054232,
  'HasAns_total': 5928,
  'NoAns_exact': 73.5071488645921,
  'NoAns_f1': 73.5071488645921,
  'NoAns_total': 5945,
  'best_exact': 68.65156236839888,
  'best_exact_thresh': 0.0,
  'best_f1': 72.64203291699968,
  'best_f1_thresh': 0.0,
  'exact': 68.64313989724585,
  'f1': 72.63361044584661,
  'total': 11873},
 {'HasAns_exact': 66.02564102564102,
  'HasAns_f1': 73.3776288290072,
  'HasAns_total': 5928,
  'NoAns_exact': 70.76534903280067,
  'NoAns_f1': 70.76534903280067,
  'NoAns_total': 5945,
  'best_exact': 68.40731070496084,
  'best_exact_thresh': 0.0,
  'best_f1': 72.07804124470282,
  'best_f1_thresh': 0.0,
  'exact': 68.3988882338078,
  'f1': 72.06961877354969,
  'total': 11873},
 {'HasAns_exact': 64.1025641025641,
  'HasAns_f1': 70.86208556046033,
  'HasAns_total': 5928,
  'NoAns_exact': 74.76871320437343,
  'NoAns_f1': 74.76871320437343,
  'NoAns_total': 5945,
  'best_exact': 69.4432746567843,
  'best_exact_thre

# RUN 8

# epochs 64-68 

In [29]:
base_path = 'drive/MyDrive/electra-small-squad_v2_run8/'
run_8_epochs = ['checkpoint-4079', 'checkpoint-8158','checkpoint-12237','checkpoint-16316','checkpoint-20395',] # from saved checkpoints 
e = 64 # track epoch being evaluated 
metric_run8=[]
for checkpoint in run_8_epochs:
  print(f'EVAL EPOCH {e}')
  e +=1
  metric_run8.append(checkpoint_eval(base_path+checkpoint))
  print(f'{len(metric_run8)} Checkpoints done')
  print('\n')
  print('\n')

EVAL EPOCH 64


Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

Didn't find file drive/MyDrive/electra-small-squad_v2_run8/checkpoint-4079/added_tokens.json. We won't load it.
loading file drive/MyDrive/electra-small-squad_v2_run8/checkpoint-4079/vocab.txt
loading file drive/MyDrive/electra-small-squad_v2_run8/checkpoint-4079/tokenizer.json
loading file None
loading file drive/MyDrive/electra-small-squad_v2_run8/checkpoint-4079/special_tokens_map.json
loading file drive/MyDrive/electra-small-squad_v2_run8/checkpoint-4079/tokenizer_config.json
loading configuration file drive/MyDrive/electra-small-squad_v2_run8/checkpoint-4079/config.json
Model config ElectraConfig {
  "_name_or_path": "drive/MyDrive/electra-small-squad_v2_run7/checkpoint-20395",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddi

  0%|          | 0/12 [00:00<?, ?ba/s]

...

  0%|          | 0/12 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `ElectraForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 11969
  Batch size = 8


Post-processing 11873 example predictions split into 11969 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

5 Checkpoints done






In [30]:
metric_run8

[{'HasAns_exact': 63.663967611336034,
  'HasAns_f1': 70.76632596838081,
  'HasAns_total': 5928,
  'NoAns_exact': 74.14634146341463,
  'NoAns_f1': 74.14634146341463,
  'NoAns_total': 5945,
  'best_exact': 68.92108144529605,
  'best_exact_thresh': 0.0,
  'best_f1': 72.46717597410621,
  'best_f1_thresh': 0.0,
  'exact': 68.91265897414301,
  'f1': 72.45875350295314,
  'total': 11873},
 {'HasAns_exact': 65.87381916329285,
  'HasAns_f1': 73.27045116740362,
  'HasAns_total': 5928,
  'NoAns_exact': 68.84777123633306,
  'NoAns_f1': 68.84777123633306,
  'NoAns_total': 5945,
  'best_exact': 67.36292428198433,
  'best_exact_thresh': 0.0,
  'best_f1': 71.05594496086671,
  'best_f1_thresh': 0.0,
  'exact': 67.36292428198433,
  'f1': 71.05594496086665,
  'total': 11873},
 {'HasAns_exact': 64.96288798920378,
  'HasAns_f1': 71.99563422491451,
  'HasAns_total': 5928,
  'NoAns_exact': 72.9857022708158,
  'NoAns_f1': 72.9857022708158,
  'NoAns_total': 5945,
  'best_exact': 68.9800387433673,
  'best_exact_